# Compute Dose for a Single Aperture

This example computes the dose in the patient's body from the first control point in the first
field of the DICOM plan.

To run this example, you will need:
- A DICOM RT file
- An external surface mesh, in STL or OBJ format
- Finite Pencil Beam kernel data

In [1]:
using Pkg; Pkg.activate(".")
using DoseCalculations
using SparseArrays

  Activating project at `~/Code/DoseCalculations.jl/examples`


## Setup

Load the DICOM plan, and select the first control point from the first field

In [2]:
plan = load_dicom("path/to/dicom/RP.....dcm")
field = plan[1] # Select the first field
controlpoint = field[1] # Select the first control point

ϕg: -110.0°,θb: 45.0°,ΔMU: 2.7551672


Load the external surface mesh, and create a cylindrical external surface.
This will take a while to compute, but is faster for general mesh-ray intersections used in the dose calculation later on.

The library expects all positions to be in the IEC Fixed coordinate system.
Depending on which coordinate system your mesh data is in, you may have to transform the mesh into IEC Fixed.
In this example, the mesh is in the IEC Patient coordinate system.

In [3]:
mesh = load_structure_from_ply("path/to/body.stl")
trans = patient_to_fixed(getisocenter(controlpoint))

surf = CylindricalSurface(transform(mesh, trans));

Use the external surface to generate a set of dose points within the surface.

You may want your dose positions to not be in the IEC Fixed coordinate system for further analysis/visualisation with other tools.
This is what the `trans` argument is in the call to `DoseGridMasked`.
This will transform the external surface (which is in IEC Fixed) to whichever coordinate system you need (in this case, IEC Patient), and set your positions in that coordinate system.
However, that will mean that these positions will have to be transformed back into the IEC Fixed coordinate system.

In [4]:
pos = DoseGridMasked(5., SurfaceBounds(surf), trans)
pos_fixed = trans.(pos);

We'll setup the dose calculation algorithm, calibrating it at 100 MU = 1 Gy for 100cm² fieldsize at a source-axis distance of 1000m.

In [5]:
calc = FinitePencilBeamKernel("path/to/kernel/file.jld")
calibrate!(calc, 100., 100., 1000.);

Finally, we can set up the bixels and beamlets.
Here, we choose the bixel grid such that it spans the jaw positions

In [6]:
bixels = BixelGrid(getjaws(controlpoint), 5., 5.)
beamlets = Beamlet.(bixels, Ref(getgantry(controlpoint)));

## Dose Calculation
We then compute the fluence map from the MLC positions,

In [7]:
Ψ = fluence(bixels, getmlc(controlpoint)); # Compute the fluence
ΔMU = getΔMU(controlpoint);

Then compute the dose-fluence matrix.
We'll use a sparse matrix here as there are potentially many points which are not near the treatment field.
This saves on memory usage.

In [8]:
D = dose_fluence_matrix(SparseMatrixCSC, vec(pos), vec(beamlets), surf, calc);

To compute dose, the dose-fluence matrix and the fluence map are multiplied:

In [9]:
dose = ΔMU*D*vec(Ψ);

We can save to file for visualisation in other software, such as Slicer or Paraview:

In [10]:
# To VTK (Paraview)
write_vtk("dose", pos, "dose"=>dose)
# To NRRD (Slicer)
write_nrrd("dose.nrrd", pos, dose);